#Tokenizing Moves in Chess

As chess games are expressible as sequences of actions, they lend themselves well to language/sequence modeling as achievable with Transformer-based model architectures. I currently have in mind two ways of doing so:

* The most intuitive approach - model the game as a sequence of moves. Each "step" consists of a string which names a piece and the square it moves to. 
* A more interesting and information-rich approach - model the game as a sequence of board states. Each "step" consists of an 8x8 array which describes the position of every piece on the chessboard after the given ply. 

In this notebook, I will work on the former approach. I will attempt to build a vocabulary of all chess moves and save it to file, where it can later be used to tokenize chess games. This notebook assumes knowledge of algebraic notation and the basic rules of chess. 

##Objectives and Approach

We will first build move vocabularies stepwise from first principles, building and adding move strings to a Python list before writing them all out to file. We'll follow this rough trajectory: 

* Pawn moves (normal)
* Pawn moves (promotion)
* Knight moves
* Bishop moves 
* Rook moves
* Queen moves
* King moves 

A few notes for our approach: 
* While algebraic notation allows for the prepending of a move number and ellipses to contextualize a single move in a game (e.g. "7...e5" would indicate that Black's 7th move was a pawn push to e5), we will not add move numbers or ellipses to any of our move strings. Firstly, doing so would cause our vocabulary size to explode. Secondly, this information should be implicit in the move sequence comprising a whole game, and a Transformer model should be smart enough to learn how to handle this. An important consequence of this modeling decision is that the sequences of tokens by themselves will contain no information regarding the move number or player identity, so we will need to find a way of representing the games downstream to preserve this information if we decide to truncate games for training or use the model to evaluate the positions of games in the middle of play (i.e. not starting with White's first move). 
* We will not append qualifiers used for human analysis to moves (e.g. `??` or `!`) for similar reasons as above. 
* In many steps, we will "overgenerate" our vocabulary (i.e. create some move strings which do not correspond to legal moves or proper algebraic notation). While it's possible to create Python rules to ensure we only add legal, properly-notated moves to our final list, it's better and easier to add more strings to our vocabulary that do not ultimately get used (or will likely only appear in theoretical games/constructions, e.g. `Bd2e3`) to avoid having to use an `UNK` token whenever possible. As always, our final tokenizer will allow for `UNK` for edge cases such as corrupt PGN files with improper move strings. 
* For similar reasons as above, we will use the non-standard `++` notation for double-checks and add them to our vocabulary. 

Later, we will also try building a vocabulary from a database of chess games, akin to how vocabularies are built from text corpora for natural language models. 

Let's begin!

##Basic setup

In [1]:
#Very basic imports 
import math, random
import sys, os
import numpy as np
import pandas as pd
import sklearn
import scipy
from glob import glob
from tqdm import tqdm

In [2]:
#Set up some useful global objects
ALL_FILES = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
ALL_RANKS = ['1', '2', '3', '4', '5', '6', '7', '8']
CHECK = '+'
DOUBLECHECK = '++'
MATE = '#'
TAKES = 'x'

In [3]:
#Connect to Google Drive for writing files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Navigate to folder to read/write data
%cd /content/drive/MyDrive/CS-224N_W2023_Final_Project/

/content/drive/MyDrive/CS-224N_W2023_Final_Project


##Complete Vocabulary

We begin by creating a complete vocabulary from first principles which will include every possible legal move expressible in algebraic notation. This vocabulary will include moves which require 3 or more of the same type of piece on the board and are thus only reachable through multiple specific pawn promotions. It will also include a large number of "overgenerated" moves (e.g. movestrings which correspond to illegal or impossible moves and thus will never occur in a real chess game). In the below sections, we will also discuss the creation of a more limited vocabulary which should cover the vast majority of legal chess moves by frequency while reducing the vocabulary size significantly. 

In [5]:
#Set up the list to store all moves
complete_moves = []

###Pawn moves (normal)

Let's begin by creating all normal pawn moves. In algebraic notation, pawns are represented with the empty string, so we just need to create the coordinates for all squares not on the first or eighth ranks (where promotion happens). 

In [6]:
#Create the movelist
normal_pawn_moves = []

In [7]:
#Non-capture moves
for f in ALL_FILES:
  for r in ALL_RANKS[1:-1]:
    #Basic moves
    normal_pawn_moves.append(f'{f}{r}')

    #Checks
    normal_pawn_moves.append(f'{f}{r}{CHECK}')
    normal_pawn_moves.append(f'{f}{r}{DOUBLECHECK}')

    #Mates
    normal_pawn_moves.append(f'{f}{r}{MATE}')

Pawn captures are notated slightly differently than for other pieces, because there is no one-letter representation for the pawn. Instead, the file of the capturing pawn is used, followed by the square the pawn moves to (and captures on), e.g. `exd4`. 

We implement a method for generating these moves below. Note that some nonlegal moves are generated as a result of our modulo indexing strategy (e.g. `hxa3` - a pawn cannot travel like in a Pac-Man world); these tokens will just never be seen in real games and thus no representation will be learned for them. If they turn out to slow down training significantly downstream, we will prune them out. 

In [8]:
for i in range(8):
  for r in ALL_RANKS[1:-1]:
    #Takes to the right
    normal_pawn_moves.append(ALL_FILES[(i-1)%8]+TAKES+ALL_FILES[i]+r)
    normal_pawn_moves.append(ALL_FILES[(i-1)%8]+TAKES+ALL_FILES[i]+r+CHECK)
    normal_pawn_moves.append(ALL_FILES[(i-1)%8]+TAKES+ALL_FILES[i]+r+DOUBLECHECK)
    normal_pawn_moves.append(ALL_FILES[(i-1)%8]+TAKES+ALL_FILES[i]+r+MATE)
    
    #Takes to the left
    normal_pawn_moves.append(ALL_FILES[(i+1)%8]+TAKES+ALL_FILES[i]+r)
    normal_pawn_moves.append(ALL_FILES[(i+1)%8]+TAKES+ALL_FILES[i]+r+CHECK)
    normal_pawn_moves.append(ALL_FILES[(i+1)%8]+TAKES+ALL_FILES[i]+r+DOUBLECHECK)
    normal_pawn_moves.append(ALL_FILES[(i+1)%8]+TAKES+ALL_FILES[i]+r+MATE)

In [9]:
#Count of move strings
len(normal_pawn_moves)

576

###Pawn moves (promotion)

When a White pawn reaches the eighth rank or a Black pawn reaches the first rank, it can promote to a knight, bishop, rook or queen. We will need to build a movestring for each of these possibilities. 

In [10]:
#Create the movelist
promo_pawn_moves = []

In [11]:
#Non-capture moves
for f in ALL_FILES:
  for r in ['1', '8']:
    for piece in ['N', 'B', 'R', 'Q']:
      #Basic moves
      promo_pawn_moves.append(f'{f}{r}={piece}')

      #Checks
      promo_pawn_moves.append(f'{f}{r}={piece}{CHECK}')
      promo_pawn_moves.append(f'{f}{r}={piece}{DOUBLECHECK}')

      #Mates
      promo_pawn_moves.append(f'{f}{r}={piece}{MATE}')

In [12]:
for i in range(8):
  for r in ['1', '8']:
    for piece in ['N', 'B', 'R', 'Q']:
      #Takes to the right
      promo_pawn_moves.append(ALL_FILES[(i-1)%8]+TAKES+ALL_FILES[i]+r+'='+piece)
      promo_pawn_moves.append(ALL_FILES[(i-1)%8]+TAKES+ALL_FILES[i]+r+'='+piece+CHECK)
      promo_pawn_moves.append(ALL_FILES[(i-1)%8]+TAKES+ALL_FILES[i]+r+'='+piece+DOUBLECHECK)
      promo_pawn_moves.append(ALL_FILES[(i-1)%8]+TAKES+ALL_FILES[i]+r+'='+piece+MATE)
      
      #Takes to the left
      promo_pawn_moves.append(ALL_FILES[(i+1)%8]+TAKES+ALL_FILES[i]+r+'='+piece)
      promo_pawn_moves.append(ALL_FILES[(i+1)%8]+TAKES+ALL_FILES[i]+r+'='+piece+CHECK)
      promo_pawn_moves.append(ALL_FILES[(i+1)%8]+TAKES+ALL_FILES[i]+r+'='+piece+DOUBLECHECK)
      promo_pawn_moves.append(ALL_FILES[(i+1)%8]+TAKES+ALL_FILES[i]+r+'='+piece+MATE)

In [13]:
#Count of move strings
len(promo_pawn_moves)

768

###(Other) Piece moves

Now that pawns are done, we move on to chess pieces, each type of which can occupy every square on the board without the promotion condition. However, there are still unique cases we need to consider, namely where multiple pieces of the same type on the same file or rank can move to the designated square. In these cases, additional characters are needed in the movestring to disambiguate which piece made the move. 

###Knight moves

Knights move in an "L" shape - two squares in one cardinal direction, followed by one square orthogonal. Because of this, the disambiguation conditions for any given square should refer to up to 8 distinct squares (although the qualifier can also be just a rank or file identifier too). Because we don't care about overgenerating, we can handle this problem in a similar manner to how we did for pawn captures, and clean up the vocabulary downstream if need be - we just need to be a bit more strategic. Let's use a helper function that tells us where knights must reside in order to move to a square of interest, and then from this function we can obtain the necessary information to build our move strings. 

In [14]:
#Helper function - returns list of algebraic-notation squares given rank and file indices
def get_knight_origin_squares(file_idx, rank_idx):
  origin_squares = []
  origin_squares.append(f'{ALL_FILES[(file_idx-2)%8]}{ALL_RANKS[(rank_idx-1)%8]}')
  origin_squares.append(f'{ALL_FILES[(file_idx-2)%8]}{ALL_RANKS[(rank_idx+1)%8]}')
  origin_squares.append(f'{ALL_FILES[(file_idx-1)%8]}{ALL_RANKS[(rank_idx-2)%8]}')
  origin_squares.append(f'{ALL_FILES[(file_idx-1)%8]}{ALL_RANKS[(rank_idx+2)%8]}')
  origin_squares.append(f'{ALL_FILES[(file_idx+2)%8]}{ALL_RANKS[(rank_idx-1)%8]}')
  origin_squares.append(f'{ALL_FILES[(file_idx+2)%8]}{ALL_RANKS[(rank_idx+1)%8]}')
  origin_squares.append(f'{ALL_FILES[(file_idx+1)%8]}{ALL_RANKS[(rank_idx-2)%8]}')
  origin_squares.append(f'{ALL_FILES[(file_idx+1)%8]}{ALL_RANKS[(rank_idx+2)%8]}')
  return sorted(origin_squares)

In [15]:
#find origin squares of knights that can move to square d5
get_knight_origin_squares(3, 4)

['b4', 'b6', 'c3', 'c7', 'e3', 'e7', 'f4', 'f6']

In [16]:
#Create the movelist - start as a set and re-type later to avoid duplicate moves
knight_moves = set()

In [17]:
for f in range(8):
  for r in range(8):
    origin_squares = get_knight_origin_squares(f, r)
    for square in origin_squares:
      #Basic moves
      knight_moves.add(f'N{ALL_FILES[f]}{ALL_RANKS[r]}')
      knight_moves.add(f'Nx{ALL_FILES[f]}{ALL_RANKS[r]}')
      
      knight_moves.add(f'N{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}')
      knight_moves.add(f'N{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}')
      
      knight_moves.add(f'N{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}')
      knight_moves.add(f'N{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}')

      knight_moves.add(f'N{square[0]}{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}')
      knight_moves.add(f'N{square[0]}{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}')

      #Checks
      knight_moves.add(f'N{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      knight_moves.add(f'Nx{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      
      knight_moves.add(f'N{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      knight_moves.add(f'N{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      
      knight_moves.add(f'N{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      knight_moves.add(f'N{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')

      knight_moves.add(f'N{square[0]}{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      knight_moves.add(f'N{square[0]}{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      #Double-checks
      knight_moves.add(f'N{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      knight_moves.add(f'Nx{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      
      knight_moves.add(f'N{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      knight_moves.add(f'N{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      
      knight_moves.add(f'N{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      knight_moves.add(f'N{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')

      knight_moves.add(f'N{square[0]}{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      knight_moves.add(f'N{square[0]}{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')

      #Mates
      knight_moves.add(f'N{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      knight_moves.add(f'Nx{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      
      knight_moves.add(f'N{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      knight_moves.add(f'N{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      
      knight_moves.add(f'N{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      knight_moves.add(f'N{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')

      knight_moves.add(f'N{square[0]}{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      knight_moves.add(f'N{square[0]}{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')

In [18]:
knight_moves = list(knight_moves)

In [19]:
len(knight_moves)

8704

###Bishop moves


As bishops move diagonally, there are (for most squares on the board) a larger number of possible originating squares on which bishops could reside to move to a given square of interest. Once again, we'll create a helper function to figure out what these originating squares are and use it to build our move strings. 

In [20]:
#Helper function - returns list of algebraic-notation squares given rank and file indices
def get_bishop_origin_squares(file_idx, rank_idx):
  origin_squares = []
  #right diagonals:
  for f, i in zip(range(file_idx+1, 8), range(1, 8-file_idx)):
    r_upper = rank_idx + i
    r_lower = rank_idx - i
    if r_upper < 8: origin_squares.append(f'{ALL_FILES[f]}{ALL_RANKS[r_upper]}')
    if r_lower >= 0: origin_squares.append(f'{ALL_FILES[f]}{ALL_RANKS[r_lower]}')
  #left diagonals:
  for f in range(0, file_idx):
    r_upper = rank_idx + (file_idx - f)
    r_lower = rank_idx - (file_idx - f)
    if r_upper < 8: origin_squares.append(f'{ALL_FILES[f]}{ALL_RANKS[r_upper]}')
    if r_lower >= 0: origin_squares.append(f'{ALL_FILES[f]}{ALL_RANKS[r_lower]}')
  return sorted(origin_squares)

In [21]:
#find diagonals of square c3
get_bishop_origin_squares(2, 2)

['a1', 'a5', 'b2', 'b4', 'd2', 'd4', 'e1', 'e5', 'f6', 'g7', 'h8']

In [22]:
#Create the movelist
bishop_moves = set()

In [23]:
for f in range(8):
  for r in range(8):
    origin_squares = get_bishop_origin_squares(f, r)
    for square in origin_squares:
      #Basic moves
      bishop_moves.add(f'B{ALL_FILES[f]}{ALL_RANKS[r]}')
      bishop_moves.add(f'Bx{ALL_FILES[f]}{ALL_RANKS[r]}')
      
      bishop_moves.add(f'B{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}')
      bishop_moves.add(f'B{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}')
      
      bishop_moves.add(f'B{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}')
      bishop_moves.add(f'B{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}')

      bishop_moves.add(f'B{square[0]}{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}')
      bishop_moves.add(f'B{square[0]}{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}')

      #Checks
      bishop_moves.add(f'B{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      bishop_moves.add(f'Bx{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      
      bishop_moves.add(f'B{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      bishop_moves.add(f'B{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      
      bishop_moves.add(f'B{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      bishop_moves.add(f'B{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')

      bishop_moves.add(f'B{square[0]}{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      bishop_moves.add(f'B{square[0]}{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      #Double-checks
      bishop_moves.add(f'B{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      bishop_moves.add(f'Bx{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      
      bishop_moves.add(f'B{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      bishop_moves.add(f'B{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      
      bishop_moves.add(f'B{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      bishop_moves.add(f'B{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')

      bishop_moves.add(f'B{square[0]}{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      bishop_moves.add(f'B{square[0]}{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')

      #Mates
      bishop_moves.add(f'B{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      bishop_moves.add(f'Bx{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      
      bishop_moves.add(f'B{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      bishop_moves.add(f'B{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      
      bishop_moves.add(f'B{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      bishop_moves.add(f'B{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')

      bishop_moves.add(f'B{square[0]}{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      bishop_moves.add(f'B{square[0]}{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')

In [24]:
bishop_moves = list(bishop_moves)

In [25]:
len(bishop_moves)

11520

###Rook moves

Unlike with the knight, bishop and queen, rooks only need a rank or file value for disambiguation and never both at the same time. This is because for any given rook moving to a square of interest, it is not possible for both 1) a second rook that can also move to the square of interest which is on the same file of the first rook, AND 2) a third rook that can also move to the square of interest which is on the same rank of the first rook, to be present simultaneously. This makes our task of generating move lists much easier, and it can be done without a helper function. 

In [26]:
#Create the movelist
rook_moves = set()

In [27]:
for f in range(8):
  for r in range(8):
    for square in origin_squares:
      #define origin squares
      file_squares = list(range(0, f)) + list(range(f+1, 8))
      rank_squares = list(range(0, r)) + list(range(r+1, 8))
      
      #Basic moves
      rook_moves.add(f'R{ALL_FILES[f]}{ALL_RANKS[r]}')
      rook_moves.add(f'Rx{ALL_FILES[f]}{ALL_RANKS[r]}')
      

      #Checks
      rook_moves.add(f'R{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      rook_moves.add(f'Rx{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      #Double-checks
      rook_moves.add(f'R{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      rook_moves.add(f'Rx{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')

      #Mates
      rook_moves.add(f'R{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      rook_moves.add(f'Rx{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')

      #Origin files
      for origin_file in file_squares:
        rook_moves.add(f'R{ALL_FILES[origin_file]}{ALL_FILES[f]}{ALL_RANKS[r]}')
        rook_moves.add(f'R{ALL_FILES[origin_file]}x{ALL_FILES[f]}{ALL_RANKS[r]}')
      
        rook_moves.add(f'R{ALL_FILES[origin_file]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
        rook_moves.add(f'R{ALL_FILES[origin_file]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
        rook_moves.add(f'R{ALL_FILES[origin_file]}{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
        rook_moves.add(f'R{ALL_FILES[origin_file]}x{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      
        rook_moves.add(f'R{ALL_FILES[origin_file]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
        rook_moves.add(f'R{ALL_FILES[origin_file]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      
      #Origin ranks
      for origin_rank in rank_squares:
        rook_moves.add(f'R{ALL_RANKS[origin_rank]}{ALL_FILES[f]}{ALL_RANKS[r]}')
        rook_moves.add(f'R{ALL_RANKS[origin_rank]}x{ALL_FILES[f]}{ALL_RANKS[r]}')

        rook_moves.add(f'R{ALL_RANKS[origin_rank]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
        rook_moves.add(f'R{ALL_RANKS[origin_rank]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
        rook_moves.add(f'R{ALL_RANKS[origin_rank]}{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
        rook_moves.add(f'R{ALL_RANKS[origin_rank]}x{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')

        rook_moves.add(f'R{ALL_RANKS[origin_rank]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
        rook_moves.add(f'R{ALL_RANKS[origin_rank]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')

In [28]:
rook_moves = list(rook_moves)

In [29]:
len(rook_moves)

7680

###Queen moves

The best way to disambiguate queen moves is to start from the bishop-moves framework, extending the diagonal-finding helper function to include rook moves (along a rank or file) to create a helper function to find all origin squares for a queen on a square of interest. 

In [30]:
#Helper function - returns list of algebraic-notation squares given rank and file indices
def get_queen_origin_squares(file_idx, rank_idx):
  origin_squares = set()
  #right diagonals:
  for f, i in zip(range(file_idx+1, 8), range(1, 8-file_idx)):
    r_upper = rank_idx + i
    r_lower = rank_idx - i
    if r_upper < 8: origin_squares.add(f'{ALL_FILES[f]}{ALL_RANKS[r_upper]}')
    if r_lower >= 0: origin_squares.add(f'{ALL_FILES[f]}{ALL_RANKS[r_lower]}')
    #rightward on the same rank
    origin_squares.add(f'{ALL_FILES[f]}{ALL_RANKS[rank_idx]}')

  #left diagonals:
  for f in range(0, file_idx):
    r_upper = rank_idx + (file_idx - f)
    r_lower = rank_idx - (file_idx - f)
    if r_upper < 8: origin_squares.add(f'{ALL_FILES[f]}{ALL_RANKS[r_upper]}')
    if r_lower >= 0: origin_squares.add(f'{ALL_FILES[f]}{ALL_RANKS[r_lower]}')
    #leftward on the same rank
    origin_squares.add(f'{ALL_FILES[f]}{ALL_RANKS[rank_idx]}')
  
  #up and down on same file:
  all_ranks = list(range(0, rank_idx)) + list(range(rank_idx+1, 8))
  for rank in all_ranks:
    origin_squares.add(f'{ALL_FILES[file_idx]}{ALL_RANKS[rank]}')

  return sorted(list(origin_squares))

In [31]:
#find all legal queen moves to square c3
get_queen_origin_squares(2, 2)

['a1',
 'a3',
 'a5',
 'b2',
 'b3',
 'b4',
 'c1',
 'c2',
 'c4',
 'c5',
 'c6',
 'c7',
 'c8',
 'd2',
 'd3',
 'd4',
 'e1',
 'e3',
 'e5',
 'f3',
 'f6',
 'g3',
 'g7',
 'h3',
 'h8']

In [32]:
#Create the movelist
queen_moves = set()

In [33]:
for f in range(8):
  for r in range(8):
    origin_squares = get_queen_origin_squares(f, r)
    for square in origin_squares:
      #Basic moves
      queen_moves.add(f'Q{ALL_FILES[f]}{ALL_RANKS[r]}')
      queen_moves.add(f'Qx{ALL_FILES[f]}{ALL_RANKS[r]}')
      
      queen_moves.add(f'Q{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}')
      queen_moves.add(f'Q{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}')
      
      queen_moves.add(f'Q{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}')
      queen_moves.add(f'Q{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}')

      queen_moves.add(f'Q{square[0]}{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}')
      queen_moves.add(f'Q{square[0]}{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}')

      #Checks
      queen_moves.add(f'Q{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      queen_moves.add(f'Qx{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      
      queen_moves.add(f'Q{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      queen_moves.add(f'Q{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      
      queen_moves.add(f'Q{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      queen_moves.add(f'Q{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')

      queen_moves.add(f'Q{square[0]}{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      queen_moves.add(f'Q{square[0]}{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      #Double-checks
      queen_moves.add(f'Q{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      queen_moves.add(f'Qx{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      
      queen_moves.add(f'Q{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      queen_moves.add(f'Q{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      
      queen_moves.add(f'Q{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      queen_moves.add(f'Q{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')

      queen_moves.add(f'Q{square[0]}{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')
      queen_moves.add(f'Q{square[0]}{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{DOUBLECHECK}')

      #Mates
      queen_moves.add(f'Q{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      queen_moves.add(f'Qx{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      
      queen_moves.add(f'Q{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      queen_moves.add(f'Q{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      
      queen_moves.add(f'Q{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      queen_moves.add(f'Q{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')

      queen_moves.add(f'Q{square[0]}{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      queen_moves.add(f'Q{square[0]}{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')

In [34]:
queen_moves = list(queen_moves)

In [35]:
len(queen_moves)

20352

###King moves

The final piece! As there can only be one king on the board for each color, we do not have to deal with disambiguation; additionally, the king cannot deliver double-checks (although, by moving out of the way of an allied piece, it can still "deliver" a discovered check or checkmate). Instead, we have to add a very tiny number of hard-coded move strings for castling, the king's special move.  

In [36]:
#Create the movelist
king_moves = []

In [37]:
#All basic moves
for f in ALL_FILES:
  for r in ALL_RANKS:
    king_moves.append(f'K{f}{r}')
    king_moves.append(f'Kx{f}{r}')
    king_moves.append(f'K{f}{r}{CHECK}')
    king_moves.append(f'Kx{f}{r}{CHECK}')
    king_moves.append(f'K{f}{r}{MATE}')
    king_moves.append(f'Kx{f}{r}{MATE}')

In [38]:
#Castling
KINGSIDE = 'O-O'
QUEENSIDE = 'O-O-O'

king_moves.append(f'{KINGSIDE}')
king_moves.append(f'{KINGSIDE}{CHECK}')
king_moves.append(f'{KINGSIDE}{MATE}')

king_moves.append(f'{QUEENSIDE}')
king_moves.append(f'{QUEENSIDE}{CHECK}')
king_moves.append(f'{QUEENSIDE}{MATE}')

In [39]:
len(king_moves)

390

###Final move list and writing to file

In [40]:
#Make final movelist
complete_moves += (normal_pawn_moves + promo_pawn_moves + \
                   knight_moves + bishop_moves + rook_moves + \
                   queen_moves + king_moves)

#Count total vocab size
len(complete_moves)

49990

In [41]:
#Write moves to file, one per line 
with open('complete_move_vocab.txt', 'w') as f:
  for move in complete_moves:
    f.write(f'{move}\n')
  f.close()

##Limited Vocabulary

Our complete vocabulary contains a total of 49990 distinct move strings, including some illegal/impossible/overspecified moves as well as every possible legal move in algebraic notation. However, many of these moves only occur in very rare or theoretical board positions and thus bloat the vocab size unnecessarily for normal game representation (it's almost at the limits of vocab sizes for most large language models - for example, the HuggingFace implementation of GPT has a vocab size of 40478, and GPT-2's is 50257). Therefore, we will make a smaller vocabulary using the above work as a starting point, with the following changes:

* All pieces - double-checks will not be notated uniquely from regular (single) checks; the onus will be on downstream processing scripts to transform any differently-notated move strings to standard check notation. 
* Knight - will only include qualifiers for rank OR file, not both at once (e.g. `Nbd2` or `N3e5`). This is sufficient for representing two knights for each player, which is the case for an exceeding majority of games (three or more knights on the board for one color will probably only arise in constructed positions or rare player disrespect/bad-mannering (BMing)). 
* Bishop - no qualifiers. This is sufficient for representing moves where each player has a single light-squared/dark-square bishop, which again is the case for the vast majority of games excluding composed positions/BMing. 
* Queen - will only include qualifiers for rank OR file, not both at once (e.g. `Qee5`or `Q2f2`). This is sufficient for representing two queens for each player, which can arise in rare situations/endgames (but three or more is very uncommon). 

In [42]:
#Set up the list to store all moves
limited_moves = []

###Pawn moves

We will use the same strategies as implemented for complete vocabulary generation, with the only differences being the exclusion of unique move strings for double-checks. 

In [43]:
#Create the movelist
pawn_moves_limited = []

In [44]:
#Non-capture moves
for f in ALL_FILES:
  for r in ALL_RANKS[1:-1]:
    #Basic moves
    pawn_moves_limited.append(f'{f}{r}')

    #Checks
    pawn_moves_limited.append(f'{f}{r}{CHECK}')

    #Mates
    pawn_moves_limited.append(f'{f}{r}{MATE}')

#Capture moves
for i in range(8):
  for r in ALL_RANKS[1:-1]:
    #Takes to the right
    pawn_moves_limited.append(ALL_FILES[(i-1)%8]+TAKES+ALL_FILES[i]+r)
    pawn_moves_limited.append(ALL_FILES[(i-1)%8]+TAKES+ALL_FILES[i]+r+CHECK)
    pawn_moves_limited.append(ALL_FILES[(i-1)%8]+TAKES+ALL_FILES[i]+r+MATE)
    
    #Takes to the left
    pawn_moves_limited.append(ALL_FILES[(i+1)%8]+TAKES+ALL_FILES[i]+r)
    pawn_moves_limited.append(ALL_FILES[(i+1)%8]+TAKES+ALL_FILES[i]+r+CHECK)
    pawn_moves_limited.append(ALL_FILES[(i+1)%8]+TAKES+ALL_FILES[i]+r+MATE)

#Non-capture promotions
for f in ALL_FILES:
  for r in ['1', '8']:
    for piece in ['N', 'B', 'R', 'Q']:
      #Basic moves
      pawn_moves_limited.append(f'{f}{r}={piece}')

      #Checks
      pawn_moves_limited.append(f'{f}{r}={piece}{CHECK}')

      #Mates
      pawn_moves_limited.append(f'{f}{r}={piece}{MATE}')

#Capture promotions
for i in range(8):
  for r in ['1', '8']:
    for piece in ['N', 'B', 'R', 'Q']:
      #Takes to the right
      pawn_moves_limited.append(ALL_FILES[(i-1)%8]+TAKES+ALL_FILES[i]+r+'='+piece)
      pawn_moves_limited.append(ALL_FILES[(i-1)%8]+TAKES+ALL_FILES[i]+r+'='+piece+CHECK)
      pawn_moves_limited.append(ALL_FILES[(i-1)%8]+TAKES+ALL_FILES[i]+r+'='+piece+MATE)
      
      #Takes to the left
      pawn_moves_limited.append(ALL_FILES[(i+1)%8]+TAKES+ALL_FILES[i]+r+'='+piece)
      pawn_moves_limited.append(ALL_FILES[(i+1)%8]+TAKES+ALL_FILES[i]+r+'='+piece+CHECK)
      pawn_moves_limited.append(ALL_FILES[(i+1)%8]+TAKES+ALL_FILES[i]+r+'='+piece+MATE)

In [45]:
len(pawn_moves_limited)

1008

###Knight moves

We will use the above helper function and general implementation strategy from the complete vocabulary section, but we will create fewer moves to add to the final move list while looping over board squares. 

In [46]:
#Create the movelist
knight_moves_limited = set()

In [47]:
for f in range(8):
  for r in range(8):
    origin_squares = get_knight_origin_squares(f, r)
    for square in origin_squares:
      #Basic moves
      knight_moves_limited.add(f'N{ALL_FILES[f]}{ALL_RANKS[r]}')
      knight_moves_limited.add(f'Nx{ALL_FILES[f]}{ALL_RANKS[r]}')
      
      knight_moves_limited.add(f'N{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}')
      knight_moves_limited.add(f'N{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}')
      
      knight_moves_limited.add(f'N{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}')
      knight_moves_limited.add(f'N{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}')

      #Checks
      knight_moves_limited.add(f'N{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      knight_moves_limited.add(f'Nx{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      
      knight_moves_limited.add(f'N{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      knight_moves_limited.add(f'N{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      
      knight_moves_limited.add(f'N{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      knight_moves_limited.add(f'N{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')

      #Mates
      knight_moves_limited.add(f'N{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      knight_moves_limited.add(f'Nx{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      
      knight_moves_limited.add(f'N{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      knight_moves_limited.add(f'N{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      
      knight_moves_limited.add(f'N{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      knight_moves_limited.add(f'N{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')

In [48]:
knight_moves_limited = list(knight_moves_limited)

In [49]:
len(knight_moves_limited)

3456

###Bishop moves

Because we include no rank/file qualifiers, our implementation for generating bishop moves becomes exceedingly simple. 

In [50]:
#Create the movelist
bishop_moves_limited = set()

In [51]:
for f in range(8):
  for r in range(8):
    #Basic moves
    bishop_moves_limited.add(f'B{ALL_FILES[f]}{ALL_RANKS[r]}')
    bishop_moves_limited.add(f'Bx{ALL_FILES[f]}{ALL_RANKS[r]}')

    #Checks
    bishop_moves_limited.add(f'B{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
    bishop_moves_limited.add(f'Bx{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')

    #Mates
    bishop_moves_limited.add(f'B{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
    bishop_moves_limited.add(f'Bx{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')

In [52]:
bishop_moves_limited = list(bishop_moves_limited)

In [53]:
len(bishop_moves_limited)

384

###Rook moves

Rook moves are the same as above (still need rank/file qualifiers in many common game positions), just without double-checks. 

In [54]:
#Create the movelist
rook_moves_limited = set()

In [55]:
for f in range(8):
  for r in range(8):
    for square in origin_squares:
      #define origin squares
      file_squares = list(range(0, f)) + list(range(f+1, 8))
      rank_squares = list(range(0, r)) + list(range(r+1, 8))
      
      #Basic moves
      rook_moves_limited.add(f'R{ALL_FILES[f]}{ALL_RANKS[r]}')
      rook_moves_limited.add(f'Rx{ALL_FILES[f]}{ALL_RANKS[r]}')
      
      #Checks
      rook_moves_limited.add(f'R{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      rook_moves_limited.add(f'Rx{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')

      #Mates
      rook_moves_limited.add(f'R{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      rook_moves_limited.add(f'Rx{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')

      #Origin files
      for origin_file in file_squares:
        rook_moves_limited.add(f'R{ALL_FILES[origin_file]}{ALL_FILES[f]}{ALL_RANKS[r]}')
        rook_moves_limited.add(f'R{ALL_FILES[origin_file]}x{ALL_FILES[f]}{ALL_RANKS[r]}')
      
        rook_moves_limited.add(f'R{ALL_FILES[origin_file]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
        rook_moves_limited.add(f'R{ALL_FILES[origin_file]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      
        rook_moves_limited.add(f'R{ALL_FILES[origin_file]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
        rook_moves_limited.add(f'R{ALL_FILES[origin_file]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      
      #Origin ranks
      for origin_rank in rank_squares:
        rook_moves_limited.add(f'R{ALL_RANKS[origin_rank]}{ALL_FILES[f]}{ALL_RANKS[r]}')
        rook_moves_limited.add(f'R{ALL_RANKS[origin_rank]}x{ALL_FILES[f]}{ALL_RANKS[r]}')

        rook_moves_limited.add(f'R{ALL_RANKS[origin_rank]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
        rook_moves_limited.add(f'R{ALL_RANKS[origin_rank]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')

        rook_moves_limited.add(f'R{ALL_RANKS[origin_rank]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
        rook_moves_limited.add(f'R{ALL_RANKS[origin_rank]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')

In [56]:
rook_moves_limited = list(rook_moves_limited)

In [57]:
len(rook_moves_limited)

5760

###Queen moves

We will use the above helper function and general implementation strategy from the complete vocabulary section, but we will create fewer moves to add to the final move list while looping over board squares. 

In [58]:
#Create the movelist
queen_moves_limited = set()

In [59]:
for f in range(8):
  for r in range(8):
    origin_squares = get_queen_origin_squares(f, r)
    for square in origin_squares:
      #Basic moves
      queen_moves_limited.add(f'Q{ALL_FILES[f]}{ALL_RANKS[r]}')
      queen_moves_limited.add(f'Qx{ALL_FILES[f]}{ALL_RANKS[r]}')
      
      queen_moves_limited.add(f'Q{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}')
      queen_moves_limited.add(f'Q{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}')
      
      queen_moves_limited.add(f'Q{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}')
      queen_moves_limited.add(f'Q{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}')

      #Checks
      queen_moves_limited.add(f'Q{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      queen_moves_limited.add(f'Qx{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      
      queen_moves_limited.add(f'Q{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      queen_moves_limited.add(f'Q{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      
      queen_moves_limited.add(f'Q{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')
      queen_moves_limited.add(f'Q{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{CHECK}')

      #Mates
      queen_moves_limited.add(f'Q{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      queen_moves_limited.add(f'Qx{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      
      queen_moves_limited.add(f'Q{square[0]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      queen_moves_limited.add(f'Q{square[0]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      
      queen_moves_limited.add(f'Q{square[1]}{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')
      queen_moves_limited.add(f'Q{square[1]}x{ALL_FILES[f]}{ALL_RANKS[r]}{MATE}')

In [60]:
queen_moves_limited = list(queen_moves_limited)

In [61]:
len(queen_moves_limited)

6528

###King moves

The set of all possible king moves is identical to in our complete vocab implemenation, as no rank/file qualifiers or double-checks are possible with the king. 

In [62]:
#Easy - just do for organizational purposes
king_moves_limited = king_moves 

###Final move list and writing to file

In [63]:
#Make final movelist
limited_moves += (pawn_moves_limited + knight_moves_limited + \
                  bishop_moves_limited + rook_moves_limited + \
                  queen_moves_limited + king_moves_limited)

#Count total vocab size
len(limited_moves)

17526

In [64]:
#Write moves to file, one per line 
with open('limited_move_vocab.txt', 'w') as f:
  for move in limited_moves:
    f.write(f'{move}\n')
  f.close()

Our final movelist for the limited vocabulary consists of only 17526 distinct move strings - over 60% fewer than in the complete vocab, even with a few remaining overgenerated moves! 

##Empirical Vocabulary

One final approach we will try is to parse a large number of existing chess games and extract the move sequences from them, saving the set of unique moves to serve as a vocabulary. 

The game corpus we will use for demonstration here is the collection of 5,015,361 games played on lichess.org in the month of February 2016; however, for pursuing this vocabulary-making method for downstream purposes, we should probably utilize every file we have to avoid using `UNK` tokens as much as possible. 

###File download

In [69]:
#download file 
!wget https://database.lichess.org/standard/lichess_db_standard_rated_2016-02.pgn.zst

--2022-12-29 23:37:28--  https://database.lichess.org/standard/lichess_db_standard_rated_2016-02.pgn.zst
Resolving database.lichess.org (database.lichess.org)... 51.91.80.70, 2001:41d0:403:3446::
Connecting to database.lichess.org (database.lichess.org)|51.91.80.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 908213555 (866M) [application/octet-stream]
Saving to: ‘lichess_db_standard_rated_2016-02.pgn.zst.2’

lichess_db_standard 100%[===================>] 866.14M  27.3MB/s    in 33s     

2022-12-29 23:38:01 (26.1 MB/s) - ‘lichess_db_standard_rated_2016-02.pgn.zst.2’ saved [908213555/908213555]



In [72]:
#unzip file 
!sudo apt-get install zstd
!unzstd -d lichess_db_standard_rated_2016-02.pgn.zst

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 278 kB of archives.
After this operation, 1,141 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 zstd amd64 1.3.3+dfsg-2ubuntu1.2 [278 kB]
Fetched 278 kB in 1s (435 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletyp

In [73]:
#Check for filename
%ls

complete_move_vocab.txt                    limited_move_vocab.txt
lichess_db_standard_rated_2016-02.pgn      Untitled0.ipynb
lichess_db_standard_rated_2016-02.pgn.zst


###Approach

Let's begin by investigating the contents of the `.pgn` file. 

In [81]:
#Get 100 or so lines from the file
lines = []
line_num = 0
with open ('lichess_db_standard_rated_2016-02.pgn', 'r') as f:
  for line in f:
    if line_num == 100: break
    lines.append(line)
    line_num += 1

In [82]:
lines

['[Event "Rated Blitz tournament https://lichess.org/tournament/oSyPykCM"]\n',
 '[Site "https://lichess.org/npDpU5Pm"]\n',
 '[White "Wallerdos"]\n',
 '[Black "chilico"]\n',
 '[Result "0-1"]\n',
 '[UTCDate "2016.01.31"]\n',
 '[UTCTime "23:00:02"]\n',
 '[WhiteElo "1763"]\n',
 '[BlackElo "2010"]\n',
 '[WhiteRatingDiff "-4"]\n',
 '[BlackRatingDiff "+5"]\n',
 '[ECO "?"]\n',
 '[Opening "?"]\n',
 '[TimeControl "300+0"]\n',
 '[Termination "Abandoned"]\n',
 '\n',
 ' 0-1\n',
 '\n',
 '[Event "Rated Blitz tournament https://lichess.org/tournament/oSyPykCM"]\n',
 '[Site "https://lichess.org/ivwkDdC3"]\n',
 '[White "patatero"]\n',
 '[Black "hichamsbitri"]\n',
 '[Result "1-0"]\n',
 '[UTCDate "2016.01.31"]\n',
 '[UTCTime "23:00:02"]\n',
 '[WhiteElo "1334"]\n',
 '[BlackElo "1320"]\n',
 '[WhiteRatingDiff "+15"]\n',
 '[BlackRatingDiff "-16"]\n',
 '[ECO "A40"]\n',
 '[Opening "Queen\'s Pawn"]\n',
 '[TimeControl "300+0"]\n',
 '[Termination "Abandoned"]\n',
 '\n',
 '1. d4 1-0\n',
 '\n',
 '[Event "Rated Blitz

We see that PGN files have a set of lines, called tag pairs, which begin and end with brackets enclosing a piece of game metadata. Following the game metadata is a line break, then the sequence of moves played in the game (which may be empty - so this line may contain only the game result, as in the case of the first game), and then another line break before the next game (and the first tag pair) begins. 

Our parsing strategy will thus involve the following steps:
* Identifying the line corresponding to the sequence of moves 
* Strip and split the line (by spaces) into a list of strings 
* Use a numeric-character filter on the first charactr of each resulting string to identify moves vs move numbers/result strings
* Add moves to a set object 

###Game parsing

In [83]:
#Practice with a sample game object
game = '1. e4 c5 2. d4 Nc6 3. dxc5 g6 4. Nf3 Bg7 5. Bd3 b6 6. cxb6 Qxb6 7. O-O d6 8. c3 Nf6 9. Re1 O-O 10. h3 Bb7 11. Be3 Qxb2 12. Nbd2 Qxc3 13. Nc4 Na5 14. Rc1 Qb4 15. Rb1 Qc3 16. Bd4 Qxe1+ 17. Qxe1 Nxe4 18. Bxg7 Nxf2 19. Qxf2 Kxg7 20. Nxa5 Bxf3 21. Qxf3 Rab8 22. Rxb8 Rxb8 23. Bc4 Rc8 24. Qxf7+ Kh6 25. Be2 1-0\n'

In [84]:
#Helper function that takes in a line from file corresponding to move sequence (game)
#Adds all moves to a Set object (in-place)
def parse_game(game, moveset):
  game_components = game.strip().split()
  for elem in game_components:
    if not elem[0].isnumeric():
      moveset.add(elem)

In [85]:
#Testing helper function
game_moves = set()
parse_game(game, game_moves)
list(game_moves)

['Bxg7',
 'O-O',
 'c3',
 'Qxf7+',
 'Nf6',
 'Be2',
 'Nxe4',
 'Bd3',
 'Na5',
 'Qxf3',
 'Nxa5',
 'Nbd2',
 'dxc5',
 'Qxb6',
 'Qxf2',
 'Bg7',
 'c5',
 'Bxf3',
 'Re1',
 'd6',
 'Qxb2',
 'Nc6',
 'Rxb8',
 'Be3',
 'Qxe1+',
 'Qxe1',
 'b6',
 'g6',
 'Rc1',
 'Nxf2',
 'Rab8',
 'Bd4',
 'cxb6',
 'Rc8',
 'Kxg7',
 'Nc4',
 'Qb4',
 'Bc4',
 'Qc3',
 'h3',
 'Bb7',
 'Rb1',
 'Kh6',
 'Qxc3',
 'd4',
 'Nf3',
 'e4']

###File parsing and writing

In [86]:
#Set up move set for all 5M games
all_seen_moves = set()

In [87]:
"""Store one previous line in memory for determining identity of current line

We could do a numeric character check on line[0] (find either "1." for first 
move or the numeric character corresponding to game result, EXCEPT that some 
games with unlimited time controls may have no game result - to avoid breaking
the parser or doing more complex checking for the game result, this is the 
easiest method)"""
prev_line = ''

#Progress-tracking - every 100k games, ~50 messages total 
games_parsed = 0
print_increment = 100000

with open ('lichess_db_standard_rated_2016-02.pgn', 'r') as f:
  for line in f:
    #Find the first line break per PGN-notated game
    #If the current line starts with a bracket, we've overshot and started
    #the next game's tag pairs
    if prev_line == '\n' and line[0] != '[':
      parse_game(line, all_seen_moves)
      games_parsed += 1
      if games_parsed % print_increment == 0: print(f'{games_parsed} parsed!')
    prev_line = line

100000 parsed!
200000 parsed!
300000 parsed!
400000 parsed!
500000 parsed!
600000 parsed!
700000 parsed!
800000 parsed!
900000 parsed!
1000000 parsed!
1100000 parsed!
1200000 parsed!
1300000 parsed!
1400000 parsed!
1500000 parsed!
1600000 parsed!
1700000 parsed!
1800000 parsed!
1900000 parsed!
2000000 parsed!
2100000 parsed!
2200000 parsed!
2300000 parsed!
2400000 parsed!
2500000 parsed!
2600000 parsed!
2700000 parsed!
2800000 parsed!
2900000 parsed!
3000000 parsed!
3100000 parsed!
3200000 parsed!
3300000 parsed!
3400000 parsed!
3500000 parsed!
3600000 parsed!
3700000 parsed!
3800000 parsed!
3900000 parsed!
4000000 parsed!
4100000 parsed!
4200000 parsed!
4300000 parsed!
4400000 parsed!
4500000 parsed!
4600000 parsed!
4700000 parsed!
4800000 parsed!
4900000 parsed!
5000000 parsed!


In [88]:
all_seen_moves = list(all_seen_moves)

###

In [89]:
len(all_seen_moves)

38238

In [90]:
#Write moves to file, one per line 
with open('lichess_2016-02_move_vocab.txt', 'w') as f:
  for move in all_seen_moves:
    f.write(f'{move}\n')
  f.close()

###Investigating first attempt

We got a LOT of distinct moves - 38238 - which seems surprising, given our expectation about the commonality of different algebraic-notation moves. What's going on? Let's investigate by looking at a few of the items in our final move list. 

In [91]:
all_seen_moves[:50]

['Bc5??',
 'Rfg1+',
 'Rgxf4?!',
 '-74.1]',
 'Raxc4+',
 '-9.29]',
 'Bcd3',
 'N4b5+?!',
 'N1d2',
 '-119.65]',
 'Rgxg2#',
 'R7xf4+',
 '-36.62]',
 '-24.73]',
 'R8xg3?!',
 '-40.79]',
 'Nbd5?!',
 'Q1xf4+',
 'R4b2??',
 'Reg7?!',
 'Kc5#',
 'R8xa4+',
 '-19.51]',
 'Rhg4?',
 'R5xg4+',
 'Rhh5',
 '-120.66]',
 'Qge4?!',
 'Rec1?!',
 '-76.11]',
 '-77.6]',
 'R5g4#',
 '-62.25]',
 'R1xb5',
 'Q8b3',
 'Rfxd6+?!',
 '-92.08]',
 'Qc3?',
 '-34.2]',
 'Qhxb7#',
 'Ngxh4+?',
 '-122.46]',
 'dxe3+?!',
 '-36.21]',
 '-15.13]',
 'Qxc8+',
 '-15.03]',
 'Raxa4#',
 '-65.37]',
 '-54.24]']

Ah! In the lichess database, about 6% of games have evaluations given by a chess engine. As a result, the PGN notation of moves for these games will contain additional elements besides move number, move, and final game result - there are comments denoting the centipawn advantage evaluation of a position which take on scalar values, and the minus-sign for evaluations in Black's favor escape our simple numeric-character filter. Additionally, some moves have a suffix consisting of question-marks and/or exclamation-marks denoting the quality of a move (ranging from blunder to brilliancy), which increases the size of our vocabulary without changing the identity of the move itself. Let's deal with both of these in an updated game-parsing function. 

###Updated game and file parsing

In [93]:
#Helper function that takes in a line from file corresponding to move sequence (game)
#Adds all moves to a Set object (in-place), after additional filtering
#for centipawn evaluations or move quality annotations
def parse_game_updated(game, moveset):
  game_components = game.strip().split()
  for elem in game_components:
    if elem[0] not in '0123456789-':
      elem_cleaned = elem[:elem.find('?')]
      elem_cleaned = elem[:elem.find('!')]
      moveset.add(elem_cleaned)

In [94]:
#Set up move set for all 5M games
all_seen_moves = set()

In [95]:
prev_line = ''

#Progress-tracking - every 100k games, ~50 messages total 
games_parsed = 0
print_increment = 100000

with open ('lichess_db_standard_rated_2016-02.pgn', 'r') as f:
  for line in f:
    #Find the first line break per PGN-notated game
    #If the current line starts with a bracket, we've overshot and started
    #the next game's tag pairs
    if prev_line == '\n' and line[0] != '[':
      parse_game_updated(line, all_seen_moves)
      games_parsed += 1
      if games_parsed % print_increment == 0: print(f'{games_parsed} parsed!')
    prev_line = line

100000 parsed!
200000 parsed!
300000 parsed!
400000 parsed!
500000 parsed!
600000 parsed!
700000 parsed!
800000 parsed!
900000 parsed!
1000000 parsed!
1100000 parsed!
1200000 parsed!
1300000 parsed!
1400000 parsed!
1500000 parsed!
1600000 parsed!
1700000 parsed!
1800000 parsed!
1900000 parsed!
2000000 parsed!
2100000 parsed!
2200000 parsed!
2300000 parsed!
2400000 parsed!
2500000 parsed!
2600000 parsed!
2700000 parsed!
2800000 parsed!
2900000 parsed!
3000000 parsed!
3100000 parsed!
3200000 parsed!
3300000 parsed!
3400000 parsed!
3500000 parsed!
3600000 parsed!
3700000 parsed!
3800000 parsed!
3900000 parsed!
4000000 parsed!
4100000 parsed!
4200000 parsed!
4300000 parsed!
4400000 parsed!
4500000 parsed!
4600000 parsed!
4700000 parsed!
4800000 parsed!
4900000 parsed!
5000000 parsed!


In [96]:
all_seen_moves = sorted(list(all_seen_moves))

In [97]:
len(all_seen_moves)

14100

In [98]:
#Write moves to file, one per line 
with open('lichess_2016-02_move_vocab_cleaned.txt', 'w') as f:
  for move in all_seen_moves:
    f.write(f'{move}\n')
  f.close()

This is a much more reasonable number of moves.

However, upon examining the file, a few issues still remain! 

* Mate evaluations - starting with character `#`
* Some moves still appearing with `?` character at end 

These can be fixed with a final version of the game-parsing function.

###Updated game and file parsing - final version

In [99]:
#Helper function that takes in a line from file corresponding to move sequence (game)
#Adds all moves to a Set object (in-place), after additional filtering
#for centipawn evaluations or move quality annotations
def parse_game_updated_v2(game, moveset):
  game_components = game.strip().split()
  for elem in game_components:
    if elem[0] not in '0123456789-#':
      elem_cleaned = elem[:elem.find('?')]
      elem_cleaned = elem[:elem.find('!')]
      elem_cleaned = elem[:elem.find('?')]
      moveset.add(elem_cleaned)

In [100]:
#Set up move set for all 5M games
all_seen_moves = set()

In [101]:
prev_line = ''

#Progress-tracking - every 100k games, ~50 messages total 
games_parsed = 0
print_increment = 100000

with open ('lichess_db_standard_rated_2016-02.pgn', 'r') as f:
  for line in f:
    #Find the first line break per PGN-notated game
    #If the current line starts with a bracket, we've overshot and started
    #the next game's tag pairs
    if prev_line == '\n' and line[0] != '[':
      parse_game_updated_v2(line, all_seen_moves)
      games_parsed += 1
      if games_parsed % print_increment == 0: print(f'{games_parsed} parsed!')
    prev_line = line

100000 parsed!
200000 parsed!
300000 parsed!
400000 parsed!
500000 parsed!
600000 parsed!
700000 parsed!
800000 parsed!
900000 parsed!
1000000 parsed!
1100000 parsed!
1200000 parsed!
1300000 parsed!
1400000 parsed!
1500000 parsed!
1600000 parsed!
1700000 parsed!
1800000 parsed!
1900000 parsed!
2000000 parsed!
2100000 parsed!
2200000 parsed!
2300000 parsed!
2400000 parsed!
2500000 parsed!
2600000 parsed!
2700000 parsed!
2800000 parsed!
2900000 parsed!
3000000 parsed!
3100000 parsed!
3200000 parsed!
3300000 parsed!
3400000 parsed!
3500000 parsed!
3600000 parsed!
3700000 parsed!
3800000 parsed!
3900000 parsed!
4000000 parsed!
4100000 parsed!
4200000 parsed!
4300000 parsed!
4400000 parsed!
4500000 parsed!
4600000 parsed!
4700000 parsed!
4800000 parsed!
4900000 parsed!
5000000 parsed!


In [102]:
all_seen_moves = sorted(list(all_seen_moves))

In [103]:
len(all_seen_moves)

8942

In [104]:
#Write moves to file, one per line 
with open('lichess_2016-02_move_vocab_cleaned_v2.txt', 'w') as f:
  for move in all_seen_moves:
    f.write(f'{move}\n')
  f.close()